<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/augmentation/ate/ate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import random
import pickle

In [ ]:
ate_folder = '/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/'
testset = pd.read_json(f'{ate_folder}/ATE_ABSITA_test_set/ate_absita_gold.ndjson',
                       lines=True)

In [ ]:
testset.drop(columns=['id_sentence'], inplace=True)

#Augmentation

In [ ]:
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

constraints = [RepeatModification(), StopwordModification()]

##WordNet

In [ ]:
from textattack.transformations import WordSwapWordNet
from textattack.augmentation import Augmenter

random.seed(33)

transformation = WordSwapWordNet(language='ita')

my_items = []
num_tras_to_add = 1

augmenter = Augmenter(transformation=transformation,
                      constraints=constraints,
                      pct_words_to_swap=0.5,  # Oppure 0.05
                      transformations_per_example=num_tras_to_add)

for row in testset.itertuples():
  result=augmenter.augment(row.sentence)
  for i in range(num_tras_to_add):
    ### QUI MODIFICHE
    my_items.append({'sentence': result[i],
                     'score': row.score,
                     'polarities': row.polarities,
                     'aspects_position': row.aspects_position,
                     'aspects': row.aspects})

In [ ]:
pd.DataFrame(data=my_items, columns=testset.columns).to_csv(f'{ate_folder}/ATE_ABSITA_test_set/sinonimi.csv', index=False)


##WordEmbedding

In [ ]:
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/index2word.pkl", 'rb') as output:
  i2w = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/nn.pkl", 'rb') as output:
  nn_matrix = pickle.load(output)
with open("/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

In [ ]:
import textattack

embedding = textattack.shared.WordEmbedding(embedding_matrix, w2i, i2w, nn_matrix)

In [ ]:
from textattack.transformations import WordSwapEmbedding
from textattack.augmentation import Augmenter
random.seed(33)

transformation = WordSwapEmbedding(20, embedding)

my_items = []
num_tras_to_add = 1

augmenter = Augmenter(transformation=transformation,
                      constraints=constraints,
                      pct_words_to_swap=0.5,  # Oppure 0.05
                      transformations_per_example=num_tras_to_add)

for row in testset.itertuples():
  result=augmenter.augment(row.sentence)
  for i in range(num_tras_to_add):
    ### QUI MODIFICHE
    my_items.append({'sentence': result[i],
                     'score': row.score,
                     'polarities': row.polarities,
                     'aspects_position': row.aspects_position,
                     'aspects': row.aspects})

In [ ]:
pd.DataFrame(data=my_items, columns=testset.columns).to_csv(f'{ate_folder}/ATE_ABSITA_test_set/embedding.csv', index=False)